In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA


from sklearn.model_selection import train_test_split
from sklearn import linear_model, preprocessing



In [2]:

df_de =pd.read_excel('../data/data_for_training_preprocessing.xlsx',sheet_name='sheet3',index_col=0, engine='openpyxl')
df_oog =pd.read_excel('../data/data_for_training_preprocessing.xlsx',sheet_name='sheet4',index_col=0, engine='openpyxl')
df_sf6 =pd.read_excel('../data/data_for_training_preprocessing.xlsx',sheet_name='sheet5',index_col=0, engine='openpyxl')
df_ldc =pd.read_excel('../data/data_for_training_preprocessing.xlsx',sheet_name='sheet6',index_col=0, engine='openpyxl')


In [11]:
df_de.columns

Index(['Facility_Name', 'City', 'State', 'Zip Code', 'Address', 'County',
       'Latitude', 'Longitude', 'Primary_NAICS_Code', 'IT_subparts',
       'IT_sectors', '2020', '2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011', '2020_perc_change', '2020_change_pattern',
       '2012_perc_change', '2012_change_pattern', '2013_perc_change',
       '2013_change_pattern', '2014_perc_change', '2014_change_pattern',
       '2015_perc_change', '2015_change_pattern', '2016_perc_change',
       '2016_change_pattern', '2017_perc_change', '2017_change_pattern',
       '2018_perc_change', '2018_change_pattern', '2019_perc_change',
       '2019_change_pattern'],
      dtype='object')

In [27]:
df_sf6.columns

Index(['FRD_id', 'Facility_Name', 'City', 'State', 'Zip_Code', 'Address',
       'County', 'Latitude', 'Longitude', 'Primary_NAICS_Code', 'IT_subparts',
       '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012',
       '2011', '2012_perc_change', '2012_change_pattern', '2013_perc_change',
       '2013_change_pattern', '2014_perc_change', '2014_change_pattern',
       '2015_perc_change', '2015_change_pattern', '2016_perc_change',
       '2016_change_pattern', '2017_perc_change', '2017_change_pattern',
       '2018_perc_change', '2018_change_pattern', '2019_perc_change',
       '2019_change_pattern', '2020_perc_change', '2020_change_pattern'],
      dtype='object')

In [28]:
df_oog.columns

Index(['FRS_Id', 'Facility_Name', 'Basin', 'City', 'State', 'Zip_Code',
       'Address', 'County', 'Latitude', 'Longitude', 'Primary_NAICS_Code',
       'IT_subparts', '2020', 'CO2_emi_non_bio_2020', 'Meth_CH4_emi_2020',
       'N2O_emi_2020', '2019', 'CO2_emi_non_bio_2019', 'Meth_CH4_emi_2019',
       'N2O_emi_2019', '2018', 'CO2_emi_non_bio_2018', 'Meth_CH4_emi_2018',
       'N2O_emi_2018', '2017', 'CO2_emi_non_bio_2017', 'Meth_CH4_emi_2017',
       'N2O_emi_2017', '2016', 'CO2_emi_non_bio_2016', 'Meth_CH4_emi_2016',
       'N2O_emi_2016', '2015', 'CO2_emi_non_bio_2015', 'Meth_CH4_emi_2015',
       'N2O_emi_2015', '2014', 'CCO2_emi_non_bio_2014', 'Meth_CH4_emi_2014',
       'N2O_emi_2014', '2013', 'CO2_emi_non_bio_2013', 'Meth_CH4_emi_2013',
       'N2O_emi_2013', '2012', 'CO2_emi_non_bio_2012', 'Meth_CH4_emi_2012',
       'N2O_emi_2012', '2011', 'CO2_emi_non_bio_2011', 'Meth_CH4_emi_2011',
       'N2O_emi_2011', '2012_perc_change', '2012_change_pattern',
       '2013_perc_change', 

# Subsetting the datasets with the numeric columns

In [29]:
df_de_X = df_de[['Latitude', 'Longitude','2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011']]
df_sf6_X = df_sf6[['Latitude', 'Longitude','2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011']]
df_oog_X = df_oog[['Latitude', 'Longitude','2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011']]
df_ldc_X = df_ldc[['Latitude', 'Longitude','2019', '2018', '2017', '2016', '2015', '2014',
       '2013', '2012', '2011']]

# One-hot encoded for States

In [31]:
df_de_st_dummies=pd.get_dummies(df_de['State'])
df_de_X= df_de_X.join(df_de_st_dummies)
df_de_y=df_de[['2020']]
df_sf6_st_dummies=pd.get_dummies(df_sf6['State'])
df_sf6_X = df_sf6_X.join(df_sf6_st_dummies)
df_sf6_y=df_sf6[['2020']]
df_oog_st_dummies=pd.get_dummies(df_oog['State'])
df_oog_X = df_oog_X.join(df_oog_st_dummies)
df_oog_y=df_oog[['2020']]
df_ldc_st_dummies=pd.get_dummies(df_ldc['State'])
df_ldc_X= df_ldc_X.join(df_ldc_st_dummies)
df_ldc_y=df_ldc[['2020']]


# Train Test split of the datasets

In [36]:
df_de_X_train, df_de_X_test, df_de_y_train, df_de_y_test=train_test_split(df_de_X, df_de_y, test_size = 0.25, random_state = 246)
df_sf6_X_train, df_sf6_X_test, df_sf6_y_train, df_sf6_y_test=train_test_split(df_sf6_X, df_sf6_y, test_size = 0.25, random_state = 246)
df_oog_X_train, df_oog_X_test, df_oog_y_train, df_oog_y_test=train_test_split(df_oog_X, df_oog_y, test_size = 0.25, random_state = 246)
df_ldc_X_train, df_ldc_X_test, df_ldc_y_train, df_ldc_y_test=train_test_split(df_ldc_X, df_ldc_y, test_size = 0.25, random_state = 246)


# Standardizing the numeric columns

In [37]:
scaler = preprocessing.StandardScaler().fit(df_de_X_train)
df_de_X_train_scaled=scaler.transform(df_de_X_train)
df_de_X_test_scaled=scaler.transform(df_de_X_test)
scaler = preprocessing.StandardScaler().fit(df_sf6_X_train)
df_sf6_X_train_scaled=scaler.transform(df_sf6_X_train)
df_sf6_X_test_scaled=scaler.transform(df_sf6_X_test)
scaler = preprocessing.StandardScaler().fit(df_oog_X_train)
df_oog_X_train_scaled=scaler.transform(df_oog_X_train)
df_oog_X_test_scaled=scaler.transform(df_oog_X_test)
scaler = preprocessing.StandardScaler().fit(df_ldc_X_train)
df_ldc_X_train_scaled=scaler.transform(df_ldc_X_train)
df_ldc_X_test_scaled=scaler.transform(df_ldc_X_test)